In [6]:
from rag_objects.RAG_system_A import RAGSystemA
from rag_objects.RAG_system_B import RAGSystemB

In [11]:
# File paths
index_path = "embeddings/all-MiniLM-L6-v2_faiss_index.index"
embedding_model_name = "sentence-transformers/all-MiniLM-L6-v2"
gemini_token_file = "API_tokens/gemini.txt"

# Initialize the system
rag_system = RAGSystemA(index_path=index_path, embedding_model_name=embedding_model_name, gemini_token_file=gemini_token_file)


RAGSystemA initialized and ready.


In [18]:
# Query the system
user_query = "skyscrapers, high-rises, towers,  The Hague, Den Haag, Netherlands, architecture, modern buildings, cityscapes, urban landscapes,  high buildings,  tall structures,  The Hague skyscrapers"
max_text = 150
top_k = 16

result = rag_system.query(user_query=user_query, max_text=max_text, top_k=top_k, summarize=False)

# print("\nLocations Data:")
# for location in result["locations"]:
#     print(location)

# print("\nGenerated Response:")
# print(result["response"])

# print("\nPerformance Metrics:")
# print(f"Retrieval Time: {result['retrieval_time']:.4f} seconds")
# print(f"Generation Time: {result['generation_time']:.4f} seconds")
# print(f"Total Time: {result['total_time']:.4f} seconds")
# print(f"Total Tokens Used: {result['total_tokens']}")
# print(f"Prompt Tokens Used: {result['prompt_tokens']}")
result


{'locations': [{'title': "Rijksbeschermd gezicht 's-Gravenhage - Haagse Bos",
   'main_category': 'park',
   'generated_text': "We're now standing in the Haagse Bos, a significant part of The Hague's cityscape.  This area holds a special designation: it's a nationally protected",
   'latitude': np.float64(52.090905),
   'longitude': np.float64(4.335106944444444)},
  {'title': 'Up:Town',
   'main_category': 'skyscraper',
   'generated_text': "Let's explore the Up:Town building, a striking addition to Rotterdam's skyline.  Standing at 107 meters tall, it's a significant structure on the Wijn",
   'latitude': np.float64(51.916575),
   'longitude': np.float64(4.486402777777778)},
  {'title': "Rijksbeschermd gezicht 's-Gravenhage - Willemspark",
   'main_category': 'neighbourhood',
   'generated_text': "We're now standing in the Willemspark area of The Hague, a neighbourhood with a fascinating history. This area has been officially designated as a pro",
   'latitude': np.float64(52.08618305

In [19]:
# File paths
index_path = "embeddings/all-mpnet-base-v2_faiss_index.index"
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
gemini_token_file = "API_tokens/gemini.txt"

# Initialize the system
rag_system = RAGSystemB(index_path=index_path, embedding_model_name=embedding_model_name, gemini_token_file=gemini_token_file)


RAGSystemB initialized and ready.


In [20]:
# Query the system
user_query = "What should a transit nerd see in Haarlem"
top_k = 16

result = rag_system.query(user_query=user_query,  top_k=top_k, summarize=False)

# print("\nLocations Data:")
# for location in result["locations"]:
#     print(location)

# print("\nGenerated Response:")
# print(result["response"])

# print("\nPerformance Metrics:")
# print(f"Retrieval Time: {result['retrieval_time']:.2f} seconds")
# print(f"Generation Time: {result['generation_time']:.2f} seconds")
# print(f"Total Time: {result['total_time']:.2f} seconds")
# print(f"Total Tokens Used: {result['total_tokens']}")
# print(f"Prompt Tokens Used: {result['prompt_tokens']}")
result


{'locations': [{'title': 'Koningstraat (Haarlem)',
   'main_category': 'street',
   'generated_text': "Let's explore the Koningstraat, a charming street in Haarlem's city center.  It forms a vital link between the Grote Markt, the main square, and the G",
   'latitude': np.float64(52.38033611111111),
   'longitude': np.float64(4.634022222222222)},
  {'title': 'Haarlem raccordement VAM',
   'main_category': 'station',
   'generated_text': "This Haarlem rail connection, or Hlmvam for short, is located just east of Haarlem station, on the Amsterdam-Haarlem line.  It's a fascinating example",
   'latitude': np.float64(52.382777777777775),
   'longitude': np.float64(4.658888888888889)},
  {'title': 'Station Haarlem Bolwerk',
   'main_category': 'station',
   'generated_text': "This location holds a fascinating piece of Haarlem's railway history.  Imagine this spot, near the vibrant Kinderhuissingel, where, from 1881 to 1886,",
   'latitude': np.float64(52.388305555555554),
   'longitude': n

In [12]:
df = pd.read_csv('nl_full_all_columns.csv')

In [89]:
df[df['title'].str.contains("Lakenhal")]

,Unnamed: 0,id,title,type,region,latitude,longitude,clean_text,geohash,country_code,thumbnail_link,thumbnail_file_name,thumbnail_author,thumbnail_license,thumbnail_file_extension,generated_text,category_output,level,main_category,subcategories
6341,6341,549276,Museum De Lakenhal,landmark,NL,52.163056,4.4875,Museum De Lakenhal is sinds 1874 het gemeentel...,u170wpg,NL,https://upload.wikimedia.org/wikipedia/commons...,De_Lakenhal_Leiden.jpg,Michiel1972,CC BY-SA 3.0,jpg,"Step into Museum De Lakenhal, Leiden's municip...","[1, museum, [art, history]]\n",1.0,museum,"art, history"


# Testing pipeline
## Test definitions

In [4]:
import pandas as pd
import time


In [5]:

test_cases = [
    {"user_query": "skyscrapers in the Hague", "golden_route": ["Hoftoren", "Het Strijkijzer", "De Kroon (woontoren)","Castalia (gebouw)","Zurichtoren"]},
    {"user_query": "What should a museum nerd see in Haarlem", "golden_route": ["Frans Hals Museum", "Verwey Museum Haarlem", "Teylers Museum","Het Dolhuys", "Archeologisch Museum Haarlem"]},
    {"user_query": "I like metro systems. Which stations are interesting in Amsterdam?", "golden_route": ["Nieuwmarkt (metrostation)", "Rokin (metrostation)", "Vijzelgracht (metrostation)","Weesperplein (metrostation)", "Noorderpark (metrostation)"]},
    {"user_query": "brutalist architecture Amsterdam", "golden_route": ["Hoofdgebouw Vrije Universiteit", "Leeuwenburg (Amsterdam)", "Louwesweg","Kraanspoor", "Weteringschans 26-28"]},
    {"user_query": "I want to see the most famous bridges of Amsterdam", "golden_route": ["Oudekerksbrug", "Blauwbrug", "Aluminiumbrug","Torensluis", "Sint Antoniessluishoogwaterkering"]},
    {"user_query": "the palaces of the Hague", "golden_route": ["Paleis Kneuterdijk", "Paleis Noordeinde", "Mauritshuis","Paleis Huis ten Bosch", "Vredespaleis"]},
    {"user_query": "What to see in Amsterdam to learn about the jewish heritage", "golden_route": ["Jodenbuurt (Amsterdam)", "Anne Frank Huis", "Nationaal Holocaustmuseum","Holocaust Namenmonument", "Portugees-Israëlietische Synagoge"]},
    {"user_query": "What should Rembrandt lover see in Leiden?", "golden_route": ["Latijnse school (Leiden)", "Rembrandtbrug", "Pieterskerk (Leiden)","Langebrug (Leiden)", "Museum De Lakenhal"]}
]
# test_cases = [
#     {"user_query": "skyscrapers in the Hague", "golden_route": ["Hoftoren", "Het Strijkijzer", "De Kroon (woontoren)","Castalia (gebouw)","Zurichtoren"]},
#     {"user_query": "What should a museum nerd see in Haarlem", "golden_route": ["Frans Hals Museum", "Verwey Museum Haarlem", "Teylers Museum","Het Dolhuys", "Archeologisch Museum Haarlem"]}
# ]

def evaluate_rag_system(rag_system, test_cases, rag_system_name, embedding_model_name):
    """
    Evaluate a RAG system by running multiple test cases and computing retrieval metrics.

    Args:
        rag_system (RAGSystemA): The RAG system to test.
        test_cases (list of dict): Each dict should contain:
            - 'user_query': The query string.
            - 'golden_route': List of ideal location titles.
        rag_system_name (str): Name of the RAG system being tested.
        embedding_model_name (str): Name of the embedding model used.

    Returns:
        pd.DataFrame: DataFrame containing results for all test cases.
    """
    results = []
    
    for test_case in test_cases:
        user_query = test_case["user_query"]
        golden_route = test_case["golden_route"]
        
        # Run query
        result = rag_system.query(user_query=user_query, top_k=5)
        
        # Extract retrieved titles
        retrieved_titles = [loc["title"] for loc in result.get("locations", [])]
        
        # Compute golden route metrics
        retrieved_set = set(retrieved_titles)
        golden_set = set(golden_route)
        common = retrieved_set.intersection(golden_set)
        precision = len(common) / len(retrieved_titles) if retrieved_titles else 0
        recall = len(common) / len(golden_set) if golden_set else 0
        num_retrieved_from_golden = len(common)
        
        # Store results dynamically including all dictionary keys
        result_entry = {
            "rag_system": rag_system_name,
            "embedding_model": embedding_model_name,
            "query": user_query,
            "retrieved_titles": retrieved_titles,
            "golden_route": golden_route,
            "precision": precision,
            "recall": recall,
            "num_retrieved_from_golden": num_retrieved_from_golden,
        }
        
        # Add all key-value pairs dynamically
        for key, value in result.items():
            if key not in result_entry:
                result_entry[key] = value
        
        results.append(result_entry)
    
    return pd.DataFrame(results)

## Running tests

In [6]:
from rag_objects.RAG_system_A import RAGSystemA

# Define embedding models and corresponding index files
embedding_models = {
    "sentence-transformers/all-MiniLM-L6-v2": "embeddings/all-MiniLM-L6-v2_faiss_index.index",
    "sentence-transformers/all-mpnet-base-v2": "embeddings/all-mpnet-base-v2_faiss_index.index"
}

# Initialize RAG system
gemini_token_file = "API_tokens/gemini.txt"
all_results = []

for model_name, index_path in embedding_models.items():
    print(f"Evaluating with embedding model: {model_name}")
    
    rag_system = RAGSystemA(index_path=index_path, embedding_model_name=model_name, gemini_token_file=gemini_token_file)
    
    # Run evaluation and collect results
    df_results = evaluate_rag_system(rag_system, test_cases, "RAGSystemA", model_name)
    all_results.append(df_results)

# Combine all results into a single DataFrame
final_results_df = pd.concat(all_results, ignore_index=True)
final_results_df.to_csv("rag_results.csv", index=False)

print("Evaluation completed. Results saved to 'rag_results.csv'")

Evaluating with embedding model: sentence-transformers/all-MiniLM-L6-v2
RAGSystemA initialized and ready.
Evaluating with embedding model: sentence-transformers/all-mpnet-base-v2
RAGSystemA initialized and ready.
Evaluation completed. Results saved to 'rag_results.csv'
